
# **Previsão de sobrevivência no Titanic.**
Na primeira etapa desse notebook será feita uma análise exploratória dos dados, no que seria uma etapa anterior ao processo de previsão, para determinação das features (variáveis) mais relevantes na sobrevivência dos passageiros. Posteriormente, alguns modelos serão ajustados para a previsão da sobrevivência.

In [ ]:
# Importando as bibliotecas necessárias para análise dos dados
import pandas as pd # Criação e manipulação de dataset
import numpy as np # Manipulação de matrizes
import matplotlib.pyplot as plt # Plotagem de dados
import seaborn as sns # Plotagem e visualização dos dados

In [ ]:
# Criando dataset
df = pd.read_csv("../input/titanic/train.csv")

# Visualização inicial do dataset
print(f'Número de Linhas e Colunas: {df.shape}')
df.head()

In [ ]:
# Avaliar a distribuição da classe Target
df.Survived.value_counts(normalize=True)*100

Á classe alvo está desbalanceada, mas não de forma severa. O impacto disso pode ser avaliado através da matriz de confusão, despois da previsão.

In [ ]:
# Alterar o nome das colunas para facilitar o entendimento 
df.columns = ["Id", "Sobreviveu", "Classe", "Nome", "Sexo", "Idade", "Familiares", "Dependentes", "Ticket", "Preço da Passagem", "Cabine", "Local de Embarque"]
df.head()

### **DICIONÁRIO DE DADOS:** 
**ID**: Identificador do Passageiro (int)

**Sobreviveu**: Se o passageiro sobreviveu (0 = Não, 1 = Sim)

**Classe**: Classe na qual o passageiro viajou. (1 = Primeira Classe, 2 = Segunda Classe, 3 = Terceira Classe)

**Nome**: Nome do passageiro, com o pronome de tratamento

**Sexo**: Sexo do passageiro (Male, Female)

**Idade**: Idade do passageiro (float)

**Familiares**: Quantidade de **irmãos** e **cônjuges** relativos ao passageiro

**Dependentes**: Quantidade de **filhos** ou **pais (idosos)** relativos ao passageiro

**Ticket**: Número do bilhete do passageiro

**Preço da Passagem**: Valor pago pelo passageiro para viajar (Dólar)

**Cabine**: Número da Cabine do passageiro

**Local de Embarque**: Foram 3 paradas antes de seguir para Nova York (C = Cherbourg, Q = Queenstown, S = Southampton)

In [ ]:
# Avaliação Geral dos Dados
df.describe()

Analisando o quadro de resumo acima podemos perceber algumas características gerais dos passageiros do Titanic:

1) Haviam no navio desde bebês, com menos de um ano de vida, até idosos, sendo que o passageiro mais velho tinha 80 anos. Mas em geral os passageiros eram mais jovens, sendo a **média de idade de 29,7 anos**, e **75% dos passageiros** tendo **até 38 anos**.

2) O preço da passagem variou bastante, provavelmente em função da classe em que o passageiro viajava. O valor médio da passagem paga para viajar no Titanic foi de **32,20 dólares**, mas **75% dos passageiros** pagaram até 31 dólares. Os bilhetes mais caros custaram 512 dólares.

3) A grande maioria dos passageiros viajava sem familiares (marido, esposa ou irmão) e não tinham dependentes (filhos e pais idosos).

In [ ]:
# Avaliando valores nulos
df.isnull().sum()

In [ ]:
# Proporção de passageiros que não tem Cabine
(df.Cabin.isnull().sum())/df.shape[0]

77% dos passageiros estão sem o registro da cabine. Pode ser avaliada a exclusão dessa coluna e o teste do modelo, para definir se impacta muito na previsão.

In [ ]:
# Proporção de passageiros sem Idade
(df.Age.isnull().sum())/df.shape[0]

20% dos passageiros estão sem a idade. Uma forma de corrigir esse dado é substituir a Idade pela média das idades, ou uma média para homens e uma média para mulheres. Vamos avaliar se a média da idade por sexo é diferente.

In [ ]:
# Avaliar as médias das Idades para Homens e Mulheres
df.groupby('Sex')['Age'].mean()

In [ ]:
# Avaliar o Desvio Padrão da Idade por Sexo
df.groupby('Sex')['Age'].std()

Será adotada a Idade Médica do respectivo sexo para os passageiros com dados faltantes.

Muitos modelos, para funcionarem de forma adequada, têm como premissa a independência das features (o que quase nunca ocorre no mundo real). Dessa forma, vale a pena fazer uma análise de correlação e, se possível, reduzir a dimencionalidade do dataset com a exclusão de features.

# CORRIGIR A ANÁLISE A PARTIR DESSE PONTO!

In [ ]:
# Análise de correlação entre as variáveis
df.corr().style.format('{:.2}').background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

Podemos perceber que a coluna **Local de Embarque** tem uma correlação considerável com a coluna **Classe** e correlação muito baixa com a **Target**. Por isso, essa coluna será excluída. Outro ponto é a correlção forte entre as colunas **Familiares** e **Dependentes**. Como a coluna **Dependentes** tem mais correlação com a **Target**, a coluna **Familiares** será excluída.

In [ ]:
# Deletar as colunas Familiares e Local de Embarque.
df.drop(columns=['Familiares', 'Local de Embarque'], inplace=True)
print(f'Número de Linhas e Colunas: {df.shape}')
df.head()

As colunas que apresentaram maior correlação com a **Target** foram: **Sexo** e **Preço da Passagem**. Vamos analisar a distribuição da Target nessas features. Geralmente, em acidentes desse tipo, o padrão Mulheres e Crianças primeiro deve ser seguido. Vamos avaliar se isso realmente ocorreu.
Para simplificar a análise, vou criar uma nova coluna para a faixa etária dos passageiros. A classificação por faixa etária pode tornar a análise mais simplificada, pois teria menos variações que os valores das idades e assim pode facilitar a percepção de padrões e correlações. Entreanto, devemos tomar bastante cuidado nessa etapa, pois a determinação das faixas etárias é bastante subjetiva (ainda mais considerando a época do acidente) e pode implicar viés à análise. 
Vou criar também uma variável para classificar o valor pago na passagem, afim de tentar classificar os passageiros entre Baixa Renda e Alta Renda.

In [ ]:
# Avaliar a distribuição dos passageiros por classe (pode ser um indicativo do valor da passagem)
df['Classe'].value_counts(normalize=True)*100

In [ ]:
# Primeiro precisamos avaliar a distribuição do valor da passagem, para determiar o limiar das classes
df['Preço da Passagem'].value_counts(normalize=True, bins=8).sort_index().plot.bar()

In [ ]:
# Criação de três faixas etárias para simplificar a análise
conditions = [
    (df['Idade'] <= 11.0),
    (df['Idade'] <= 30.0),
    (df['Idade'] > 30.0)]
choices = ['Criança','Adulto','Idoso']
df['Faixa Etária'] = np.select(conditions, choices)
df['Faixa Etária'] = df['Faixa Etária'].astype(str)

# Criação de duas faixas de renda, de acordo com o valor pago pela passagem
conditions = [
    (df['Preço da Passagem'] <= 320.0),
    (df['Preço da Passagem'] > 320.0)]
choices = ['Baixa renda', 'Alta renda']
df['Classe de Renda'] = np.select(conditions, choices)
df['Classe de Renda'] = df['Classe de Renda'].astype(str)

In [ ]:
# Avaliar se as colunas foram criadas de forma correta
df.head()

Agora vamos avaliar a distribuição da **Target** dentro das features.

In [ ]:
# Proporção de sobreviventes por Classe
sobreviventes_classe = df.copy()
sobreviventes_classe['Sobreviveu'] = df['Sobreviveu'] == 1
sns.barplot(data=sobreviventes_classe, y='Sobreviveu', x='Classe', hue="Classe")

In [ ]:
# Proporção de sobreviventes por Classe de Renda (ponto de corte em 320 dólares)
sobreviventes_renda = df.copy()
sobreviventes_renda['Sobreviveu'] = df['Sobreviveu'] == 1
sns.barplot(data=sobreviventes_renda, y='Sobreviveu', x='Classe de Renda', hue="Classe de Renda")

Fica bem claro a forte correlação entre a Renda e a probabilidade de sobrevivência, e também entre a classe do passageiro e a probabilidade de sobrevivência.
Vamos avaliar a distribuição da **Target** pelo **Sexo** dos passageiros.

In [ ]:
# Proporção de sobreviventes por Sexo
sobreviventes_sexo = df.copy()
sobreviventes_sexo['Sobreviveu'] = df['Sobreviveu'] == 1
sns.barplot(data=sobreviventes_sexo, y='Sobreviveu', x='Sexo', hue="Sexo")

Praticamente metade das mulheres que estavam no Titanic sobreviveram, enquanto pouco mais de 10% dos homens sobreviveram. Para destacar se realmente a Classe onde o passageiro viajava e o Sexo foram determinantes na sobrevivência, vamos plotar a proporção de sobreviventes por Sexo e Classe. 

In [ ]:
# Proporção de sobreviventes por Sexo e Classe
sobreviventes_sexo = df.copy()
sobreviventes_sexo['Sobreviveu'] = df['Sobreviveu'] == 1
sns.barplot(data=sobreviventes_sexo, y='Sobreviveu', x='Sexo', hue="Classe")

De acordo com o gráfico acima fica perceptível a maior proporção de sobreviventes entre as mulheres, além do impacto da classe em que o passageiro viajava na prooprção de sobrevivência. A maior discrepância pode ser observada entre mulheres da primeira e segunda classes e homens de terceira classe.

In [ ]:
# Proporção de sobreviventes por Faixa Etária (0-11, 12-30, > 30)
sobreviventes_faixa = df.copy()
sobreviventes_faixa['Sobreviveu'] = df['Sobreviveu'] == 1
sns.barplot(data=sobreviventes_faixa, y='Sobreviveu', x='Faixa Etária', hue='Sexo')

Percebe-se também a existência de correlação entre a faixa etária e a sobrevivência, onde **Crianças** e **Idosos** tiveram uma probabilidade de sobrevivência maior que os jovens. Mesmo em passageiros do sexo masculino, a sobrevivência das crianças foi maior. Pode-se perceber, portanto, que o padrão de salvamento **Mulheres e Crianças primeiro** parece ter sido utilizado no caso do Titanic. 
Existe relação também entre a **Classe** e **Renda** do passageiro e a sobrevivência, onde a primeira e segunda classe apresentaram maior probabilidade de sobrevivência. 

## **Após a análise preliminar do dataset percebe-se que os fatores mais relevantes na sobrevivênicia para os passageiros do Titanic são:** 

1) **Sexo**: onde as mulheres tiveram proporção sobrevivência muito superior aos homens.

2) **Classe**: onde passageiros da primeira e segunda classe tiveram sobrevivência superior aos passageiros da terceira classe.

3) **Idade**: onde crianças e idosos apresentaram sobrevivência maior que os jovens. 

4) **Valor pago na Passagem**: onde passageiros que pagaram mais caro apresentaram sobrevivência maior que os passageiros que pagaram mais barato na passagem.

# **Modelagem e Previsão**

Com um melhor entendimento do dataframe e de como as variáveis se relacionam com o resultado final, além de uma reestruturação dos dados, podemos passar à etapa de treinamento e teste dos modelos para tentar prever o resultado final para determinado indivíduo.
Como o dataset tem uma baixa dimencionalidade e também considerando que a relação entre as features e a target está bem estabelecida, vou treinar um modelo Decision Tree com poucas camadas, buscando tanto alta acurácia como boa interpretação das previsões.

In [ ]:
from sklearn.preprocessing import MinMaxScaler # Transformação de dados 
from sklearn.model_selection import train_test_split # Partir dados em amostras
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier # Modelo Árvore de Decisão
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score # Métodos para avaliar acurácia dos modelos

In [ ]:
# Avaliar dataset para finalizar o processamento
df.head()

In [ ]:
# Excluir as colunas Faixa Etária e Classe de Renda
df.drop(columns=['Faixa Etária', 'Classe de Renda'], inplace=True)

# Passar a coluna Classe para Categórica, para que funcionem da forma correta

df['Classe'] = df['Classe'].astype(str)

In [ ]:
# Avaliar os formatos dos dados
df.dtypes

In [ ]:
# Passar a variável categórica Classe para colunas (One-hot-encoding)
df = pd.get_dummies(df)
df.head()

In [ ]:
# Definir Variáveis de Entrada e Saída
X = df.drop('Sobreviveu', axis = 1)
y = df['Sobreviveu'] 

In [ ]:
# Definir Conjunto de Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=44)

In [ ]:
# Criar Scaler para normalizar os dados (Faz-se somente no Treino e posteriormente aplica no teste)
escala = MinMaxScaler()
X_train = escala.fit_transform(X_train)
X_test = escala.transform(X_test)

In [ ]:
# Validação Cruzada do Modelo
clf_Tree = DecisionTreeClassifier()

params = {'criterion':['gini', 'entropy'], 'max_depth':[2, 3, 4, 5, 6, 7, 8]}
grid = GridSearchCV(estimator=clf_Tree, param_grid=params, scoring='f1', cv=5)

# Training and Validating Model
grid.fit(X_train, y_train)
print(pd.DataFrame(grid.cv_results_)[['rank_test_score','mean_test_score']].sort_values('rank_test_score').head(5))
print(f'Melhores parâmetros: {grid.best_params_}')

In [ ]:
# Traning Tuning Model
tuning_clf_tree = DecisionTreeClassifier(criterion='gini', max_depth=10)
tuning_clf_tree.fit(X_train, y_train)
y_pred = tuning_clf_tree.predict(X_test)

In [ ]:
# Evaluating Model Accuracy
print(f'Acurácia do Modelo Decision Tree: {accuracy_score(y_test, y_pred)*100}')

# Evaluating Model Confusion Matrix
matriz_tree = pd.DataFrame(confusion_matrix(y_test, y_pred))
print('\nMatriz de Confusão Decision Tree') 
print(matriz_tree)

# Evaluating Model classification Report 
print('Classification Report do Modelo Decision Tree')
print (classification_report(y_test, y_pred))

# **Submissão da Previsão**

In [ ]:
# Previsão final com o modelo KNN
prev = clf_KNN.predict(x_test)
prev = prev.astype(np.int16)

In [ ]:
# Gerando CSV para submissão
sub = pd.read_csv("../input/titanic/gender_submission.csv")
sub['Survived'] = prev
sub.to_csv('submission.csv', index=False)

In [ ]:
# Arquivo final para Submissão
sub.head()

**Nenhum dos modelos teve alta acurácia, mas a análise serve para o entendimento do melhor modelo a ser utiliado no problema.**

**Contudo são necessários ajustes nos parâmetros e avaliação das features a fim de melhorar a previsão dos modelos.**

**Ainda assim, é possível prever a sobreviência ou não de certo passageiro com até 77% de certeza.**

**Críticas e contribuições são muito bem vindas!**